In [7]:
import requests
import io
import os
import re
from bs4 import BeautifulSoup
import pandas as pd
urls = ['https://fbref.com/en/squads/a2d435b3/Leicester-City', 'https://fbref.com/en/squads/47c64c55/Crystal-Palace', 'https://fbref.com/en/squads/822bd0ba/Liverpool', 'https://fbref.com/en/squads/b8fd03ef/Manchester-City', 'https://fbref.com/en/squads/cff3d9bb/Chelsea', 'https://fbref.com/en/squads/19538871/Manchester-United', 'https://fbref.com/en/squads/8cec06e1/Wolverhampton-Wanderers', 'https://fbref.com/en/squads/1df6b87e/Sheffield-United', 'https://fbref.com/en/squads/361ca564/Tottenham-Hotspur', 'https://fbref.com/en/squads/18bb7c10/Arsenal', 'https://fbref.com/en/squads/943e8050/Burnley', 'https://fbref.com/en/squads/d3fd31cc/Everton', 'https://fbref.com/en/squads/b2b47a98/Newcastle-United', 'https://fbref.com/en/squads/33c895d4/Southampton', 'https://fbref.com/en/squads/d07537b9/Brighton--Hove-Albion', 'https://fbref.com/en/squads/7c21e445/West-Ham-United', 'https://fbref.com/en/squads/2abfe087/Watford', 'https://fbref.com/en/squads/4ba7cbea/Bournemouth', 'https://fbref.com/en/squads/8602292d/Aston-Villa', 'https://fbref.com/en/squads/1c781004/Norwich-City']
os.mkdir('PlayerData')




In [8]:
for url in urls:
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.find_all('table', id = 'stats_standard_ks_3232')
    table = table[0]
    teamname = url[37:]
    
    
    for row in table.find_all('tr'):
        position = row.find_all("td", {"data-stat" : "position"})
        for pos in position:
            if (pos.text == 'FW'):
                playername = row.find("th", {"data-stat" : "player"}).text
                url2 = 'https://fbref.com'+row.find_all('a', attrs={'href': re.compile("/en/players")})[1].get('href')
                csv_filename = teamname+'-'+playername + '.csv'
                txt_filename = playername + '.txt'
    
                response2 = requests.get(url2)
                soup2 = BeautifulSoup(response2.content, 'html.parser')
                table2 = soup2.find_all('table', id = 'ks_matchlogs_3232')
                table2 = table2[0]
            
                with io.open (txt_filename, 'w', encoding="utf-8") as r:
                    r.write('Day,Round,Venue,Result,Squad,Opponent,Start,Pos,Min,Gls,Ast,PK,PKatt,Sh,SoT,CrdY,CrdR,Touches,Press,Tkl,Int,Blocks,xG,npxG,xA,SCA,GCA,Cmp,Att,Cmp%,PassPrgDist,Carries,CarryPrgDist,Succ,DribAtt,Match Report,')
                    r.write('\n')
                    
                    for row2 in table2.find_all('tr')[:-1]:
                        for cell2 in row2.find_all('td'):
                            l = len(row2.find_all('td'))
                            if (l != 36):
                                continue
                            if (cell2.text.find(',') >= 0):
                                r.write(cell2.text.replace(',',' '))
                            else:
                                r.write(cell2.text)
                        
                            r.write(',')
                        r.write('\n')
                    
                    
                df = pd.read_csv(txt_filename)
                df = df.drop(['Match Report','Unnamed: 36'], axis= 1)
                df.to_csv('PlayerData/'+csv_filename, index=None)
    
            